In [ ]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.AdvancedSolvers;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
string buildNr = System.Environment.GetEnvironmentVariable("BUILD_NUMBER");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "run" : buildname;
string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
buildNr = String.IsNullOrEmpty(buildNr)? thedate : buildNr;

string database_name = String.IsNullOrEmpty(dbname)? "Benchmark" : dbname;
string table_name = String.Concat(buildname, "_", buildNr, "_", database_name);

In [ ]:
string CurrentDocDir = Directory.GetCurrentDirectory();
string path = String.Concat(CurrentDocDir,@"/",table_name,".json");
var SesTab = TableExtensions.LoadFromFile(path);

Error: System.IO.FileNotFoundException: Could not find file 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\run_17-1-2022.json'.
File name: 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\run_17-1-2022.json'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path, Encoding encoding, Boolean detectEncodingFromByteOrderMarks)
   at System.IO.File.InternalReadAllText(String path, Encoding encoding)
   at System.IO.File.ReadAllText(String path)
   at BoSSS.Application.BoSSSpad.TableExtensions.LoadFromFile(String filePath) in B:\BoSSS

In [ ]:
//var SesTab = TableExtensions.LoadFromFile(@"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\data_weak\weak_scaling_5_1_p-MG_2021-10-13.json");
//var SesTab = TableExtensions.LoadFromFile(@"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\data_strong\strong_scaling_3_1_2021-9-24.json");

In [ ]:
public static class DoWhatUWant{
    public static PlotRowSelectorEx Solver_DG_Proj(string methodname){
        return delegate (int iSweep, int iTabRow, 
                    IDictionary<string, object> Row, 
                    out string Nmn, 
                    out PlotFormat Fmt,
                    out double xValue, out double yValue) { 

            int k           = Convert.ToInt32(Row["DGdegree:Velocity*"]);
            
            Fmt           = new PlotFormat();
            Fmt.PointSize = 0.5;
            Fmt.Style     = Styles.LinesPoints;
            Fmt.LineWidth = 3;
            
            switch(k){
            case 2:
                Nmn="k2";
                Fmt.LineColor = LineColors.Red;
                break;
            case 3:
                Nmn="k3";
                Fmt.LineColor = LineColors.Orange;
                break;
            case 4:
                Nmn="k4";
                Fmt.LineColor = LineColors.Blue;
                break;
            default:
                throw new Exception("vergiss es");
            }
            double time = Convert.ToDouble(Row[methodname]);
            double iter = Convert.ToDouble(Row["LSolverIter"]);
            double timeSchwarz = Convert.ToDouble(Row["Schwarz.Solve"]);
            //yValue = Convert.ToDouble(Row["NoOfNodes"]);
            //yValue = time/iter;
            //yValue = Convert.ToDouble(Row["MGDepth"]);
            //yValue=Convert.ToDouble(Row["LSolverIter"]);
            //yValue = iter;
            yValue = time;
            xValue = Convert.ToDouble(Row["NoOfCores"]);
        };
    }
}

In [ ]:
System.Data.DataView view = new System.Data.DataView(SesTab);
view.Sort = "NoOfCores, DGdegree:Velocity*";
SesTab=view.ToTable();

In [ ]:
int L = SesTab.Rows.Count();
int xMin = Convert.ToInt32(SesTab.ExtractColumns("NoOfCores").Rows[0].ItemArray.First());
int xMax = Convert.ToInt32(SesTab.ExtractColumns("NoOfCores").Rows[L-1].ItemArray.First());
new object[]{xMin,xMax}

index,value
0,8
1,256


In [ ]:
Func<string, Plot2Ddata[,]> GenDiagram = delegate(string methname){
    Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
    multiplots[0,0] = SesTab.ToPlot(DoWhatUWant.Solver_DG_Proj(methname),1);
    multiplots[0,0].LogX = true;
    multiplots[0,0].LogY = true;
    multiplots[0,0].tmargin = 0;
    multiplots[0,0].bmargin = 5;
    multiplots[0,0].XrangeMin=xMin;
    multiplots[0,0].XrangeMax=xMax;
    multiplots[0,0].YrangeMin=.1;
    multiplots[0,0].YrangeMax=1000000;
    multiplots[0,0].Ylabel = "iter";
    multiplots[0,0].Xlabel = "cores";
    multiplots[0,0].LegendAlignment=new string[]{"i","t","r"};
    return multiplots;
};

In [ ]:
GenDiagram("SolverRun_time").PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top right Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 k4 
 
 
 
 
 k4

In [ ]:
Func<string, int, double> AverageSlope = delegate(string methodname, int k){
    var tmpTab = SesTab.ExtractColumns(methodname,"NoOfCores","DGdegree:Velocity*");
    Func<int, IDictionary<string, object>, bool> Selector = delegate(int i, IDictionary<string, object> RowValues){
        RowValues.TryGetValue("DGdegree:Velocity*",out object kout);
        int DGdegree = Convert.ToInt32(kout);
        return DGdegree==k;
    };
    tmpTab=tmpTab.ExtractRows(Selector,true);
    var slopeList = new List<double>();
    double average = 0;
    for(int iRow = 1; iRow < tmpTab.Rows.Count(); iRow++){
        double t2 = Convert.ToDouble(tmpTab.Rows[iRow][methodname]);
        double t1 = Convert.ToDouble(tmpTab.Rows[iRow-1][methodname]);
        //t2=t2/i2;
        //t1=t1/i1;
        double c2 = Convert.ToDouble(tmpTab.Rows[iRow]["NoOfCores"]);
        double c1 = Convert.ToDouble(tmpTab.Rows[iRow-1]["NoOfCores"]);
        double slope = Math.Log(t2/t1)/Math.Log(c2/c1);
        slopeList.Add(slope);
        average += slope;
    }
    return average/(tmpTab.Rows.Count()-1);
};

In [ ]:
string Mname = "SolverRun_time";
new object[] {AverageSlope(Mname,2), AverageSlope(Mname,3), AverageSlope(Mname,4)}

index,value
0,-0
1,-0
2,0.2809156255019855


In [ ]:
GenDiagram("SolverInit_time").PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top right Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 k4 
 
 
 
 
 k4

In [ ]:
string Mname = "SolverInit_time";
new object[] {AverageSlope(Mname,2), AverageSlope(Mname,3), AverageSlope(Mname,4)}

index,value
0,-0
1,-0
2,0.2368384965255837


In [ ]:
GenDiagram("LSolverIter").PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top right Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 k4 
 
 
 
 
 k4

In [ ]:
string Mname = "LSolverIter";
new object[] {AverageSlope(Mname,2), AverageSlope(Mname,3), AverageSlope(Mname,4)}

index,value
0,-0
1,-0
2,0
